In [4]:
# google vision import
import os
import sys
from google.auth.transport.requests import Request, AuthorizedSession
from google.cloud.vision import ImageAnnotatorClient, types
from google.oauth2 import id_token, credentials, service_account
import pandas

credent = service_account.Credentials.from_service_account_file(
    'anster-1593361678608.json')

def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient(credentials=credent)

    with open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.types.Image(content=content)

    objects = client.object_localization(
        image=image).localized_object_annotations           
    return objects

In [5]:
objects = localize_objects('blue_suit2.jpg')

object_array = location_preprocess(objects)
print(object_array)

NameError: name 'location_preprocess' is not defined

In [6]:
file_name = 'blue_suit2.jpg'

objects = localize_objects(file_name) #bob的function

hex_code_array = color_detection(objects, file_name)

NameError: name 'color_detection' is not defined

### Final Function

In [8]:
def location_preprocess(objects):
    object_array = [] 
    obj_count = len(objects) #計算有幾個object
    for object_ in objects:
        object_dict = {}
        
        #取得物件和可信度
        name = object_.name
        score = object_.score
        
        #取得座標
        location = object_.bounding_poly.normalized_vertices
        location_array = [location[0].x, location[0].y , location[2].x, location[2].y]
        
        
        #回傳值處理
        object_dict['name'] = name
        object_dict['score'] = score
        object_dict['location'] = location_array
        
        object_array.append(object_dict)
    
    return object_array

def color_detection(objects, file_name):
    #把物件、可信度、物件位置抓出來

    try:
        object_array = location_preprocess(objects)

        #針對每個物件增加顏色array
        for object_ in object_array: #遍歷照片中的每個物件
            color_array = find_dominate_color(file_name, object_['location']) #取得顏色號碼陣列
            print(color_array)
            
            
        return hex_code_array
    except Exception as e:
        print(f'Error: color_detection went wrong, Exception:{e}')
        return []



### 顏色偵測

In [9]:
# 顏色偵測
from __future__ import print_function
import binascii
import struct
from PIL import Image
import numpy as np
import scipy
import scipy.misc
import scipy.cluster
from PIL import Image 
import matplotlib.pyplot as plt

In [10]:
def crop_image(file_name, location):
    
    img = Image.open(file_name)
    plt.imshow(img)
    
    plt.figure()
    plt.imshow(img) ## for testing
    
    #取得照片長寬
    width, height = img.size 
    
    #得出物件上下左右
    left = location[0] * width
    top = location[1] * height
    right = location[2] * width 
    bottom = location[3] * height
    
    #裁切
    crop_img = img.crop((left, top, right, bottom)) 

    
    plt.figure()
    plt.imshow(crop_img) ## for testing
    
    return crop_img

def find_dominate_color(file_name, location, resize=False, color_nums=5):
    NUM_CLUSTERS = color_nums
    hex_code_array = []
    
    # 照片前處理
    try: 
        im = crop_image(file_name, location)
        if resize:
            print('start image resize to 150, 150')
            im = im.resize((150, 150))      # optional, to reduce time
        ar = np.asarray(im)
        shape = ar.shape
        ar = ar.reshape(scipy.product(shape[:2]), shape[2]).astype(float)
    except Exception as e:
        print(f'Error: find_dominate_color preprocessing went wrong, exception:{e}')
        return []
    
    # 計算顏色
    try:
        codes, dist = scipy.cluster.vq.kmeans(ar, NUM_CLUSTERS)
        vecs, dist = scipy.cluster.vq.vq(ar, codes)         # assign codes
        counts, bins = scipy.histogram(vecs, len(codes))    # count occurrences

        index_max = scipy.argmax(counts)                    # find most frequent
        peak = codes[index_max]
        colour = binascii.hexlify(bytearray(int(c) for c in peak)).decode('ascii')
    except Exception as e:
        print(f'Error: find_dominate_color clustering went wrong, exception:{e}')
              
#     print('most frequent is %s (#%s)' % (peak, colour))

    # 顏色編碼
    for i in codes:
        hex_code = '#'+binascii.hexlify(bytearray(int(c) for c in i)).decode('ascii')
        color_name = code_to_color(hex_code) #編碼轉成顏色名稱
        
        color_array.append(color_name) #加入陣列
        
        color_array = color_augment(color_array) #ontology 擴增
        
    return color_array

### 顏色處理

In [11]:
#顏色處理
import webcolors
import matplotlib.colors as mc
def color_predefine():
    CSS3 = webcolors.CSS3_NAMES_TO_HEX
    CSS3_r = {}
    CSS3['brown'] = '#8B4513'
    CSS3['darkbrown'] = '#654321'
    CSS3['caramel'] = '#65350F'
    CSS3['coffee'] = '#6F4137'
    CSS3['darkcoffee'] = '#3B2F2F'
    
    for k, v in CSS3.items():
        CSS3_r[v] = k
        print(k,v)
    return CSS3, CSS3_r

CSS3, CSS3_r = color_predefine()

aliceblue #f0f8ff
antiquewhite #faebd7
aqua #00ffff
aquamarine #7fffd4
azure #f0ffff
beige #f5f5dc
bisque #ffe4c4
black #000000
blanchedalmond #ffebcd
blue #0000ff
blueviolet #8a2be2
brown #8B4513
burlywood #deb887
cadetblue #5f9ea0
chartreuse #7fff00
chocolate #d2691e
coral #ff7f50
cornflowerblue #6495ed
cornsilk #fff8dc
crimson #dc143c
cyan #00ffff
darkblue #00008b
darkcyan #008b8b
darkgoldenrod #b8860b
darkgray #a9a9a9
darkgrey #a9a9a9
darkgreen #006400
darkkhaki #bdb76b
darkmagenta #8b008b
darkolivegreen #556b2f
darkorange #ff8c00
darkorchid #9932cc
darkred #8b0000
darksalmon #e9967a
darkseagreen #8fbc8f
darkslateblue #483d8b
darkslategray #2f4f4f
darkslategrey #2f4f4f
darkturquoise #00ced1
darkviolet #9400d3
deeppink #ff1493
deepskyblue #00bfff
dimgray #696969
dimgrey #696969
dodgerblue #1e90ff
firebrick #b22222
floralwhite #fffaf0
forestgreen #228b22
fuchsia #ff00ff
gainsboro #dcdcdc
ghostwhite #f8f8ff
gold #ffd700
goldenrod #daa520
gray #808080
grey #808080
green #008000
greenye

In [12]:
def closest_color(requested_color):
    min_colors = {}
    
    #計算最相近的顏色
    try:
        rgb = webcolors.hex_to_rgb(requested_color)
        for key, value in CSS3.items():
            r_c, g_c, b_c = webcolors.hex_to_rgb(value)
            rd = (r_c - rgb[0]) ** 2
            gd = (g_c - rgb[1]) ** 2
            bd = (b_c - rgb[2]) ** 2
            min_colors[(rd + gd + bd)] = value
        nearest_color = min_colors[min(min_colors.keys())]
        name = CSS3_r[nearest_color]
        return name
    except Exception as e:
        print(f'Error: closest_color went wrong, exception {e}')
        return None
        
def code_to_color(requested_color, method:str = 'hex'):
    print(requested_color[0])
    try:
        
        if method == 'hex' and str(requested_color[0]) == '#':            
            closest_name = actual_name = CSS3_r[requested_color]
        else:
            closest_name = actual_name = webcolors.rgb_to_name(requested_color)
    except ValueError:
        closest_name = closest_color(requested_color)
        actual_name = None
    except Exception as e:
        closest_name = closest_color(requested_color)
        actual_name = None
        
    return actual_name, closest_name